# [roberta-classical-chinese-base-ud-goeswith](https://huggingface.co/KoichiYasuoka/roberta-classical-chinese-base-ud-goeswith)による句間リンク(開発中)

In [ ]:
!pip install transformers ufal.chu_liu_edmonds deplacy
class UDintertexts(object):
  def __init__(self,bert):
    from transformers import AutoTokenizer,AutoModelForTokenClassification
    self.tokenizer=AutoTokenizer.from_pretrained(bert)
    self.model=AutoModelForTokenClassification.from_pretrained(bert)
  def __call__(self,text1,text2):
    import numpy,torch,ufal.chu_liu_edmonds
    text=text1+text2
    w=self.tokenizer(text,return_offsets_mapping=True)
    v=w["input_ids"]
    n,n1,n2=len(v)-1,len(text1),len(text2)
    with torch.no_grad():
      d=self.model(input_ids=torch.tensor([v[0:i]+[self.tokenizer.mask_token_id]+v[i+1:]+[v[i]] for i in range(1,n)]))
    e=d.logits.numpy()[:,1:n,:]
    e[:,:,0]=numpy.nan
    m=numpy.full((n,n),numpy.nan)
    m[1:,1:]=numpy.nanmax(e,axis=2).transpose()
    p=numpy.zeros((n,n))
    p[1:,1:]=numpy.nanargmax(e,axis=2).transpose()
    for i in range(1,n):
      m[i,0],m[i,i],p[i,0]=m[i,i],numpy.nan,p[i,i]
    mm=numpy.full((n,n),numpy.nan)
    mm[1:n1+1,n1+1:]=m[1:n1+1,n1+1:]
    mm[n1+1:,1:n1+1]=m[n1+1:,1:n1+1]
    x,y=numpy.unravel_index(numpy.nanargmax(mm),mm.shape)
    mm=numpy.full((n,n),numpy.nan)
    mm[1:n1+1,1:n1+1]=m[1:n1+1,1:n1+1]
    mm[n1+1:,n1+1:]=m[n1+1:,n1+1:]
    mm[x,y]=m[x,y]
    print(x,y,self.model.config.id2label[p[x,y]].split("|")[-1])
    mm[:,0]=m[:,0]
    h=ufal.chu_liu_edmonds.chu_liu_edmonds(mm)[0]
    u="# text = "+text+"\n"
    v=[(s,e) for s,e in w["offset_mapping"] if s<e]
    for i,(s,e) in enumerate(v,1):
      q=self.model.config.id2label[p[i,h[i]]].split("|")
      u+="\t".join([str(i),text[s:e],"_",q[0],"_","|".join(q[1:-1]),str(h[i]),q[-1],"_","_" if i<len(v) and e<v[i][0] else "SpaceAfter=No"])+"\n"
    return u+"\n"

nlp=UDintertexts("KoichiYasuoka/roberta-classical-chinese-base-ud-goeswith")
doc=nlp("不入虎穴","不得虎子")
import deplacy
deplacy.serve(doc,port=None)